In [2]:
import sys
sys.path.append("..") #Parent 
sys.path.append("../..") #grandparent
import torch
from models.components.ldm.denoiser import UNetModel
from Training_LDM.DownscalingDataModule import DownscalingDataModule
import random


In [3]:
from models.components.unet import DownscalingUnetLightning 
from models.ae_module import AutoencoderKL
from models.components.ae import SimpleConvEncoder, SimpleConvDecoder
from models.ldm_module import LatentDiffusion

In [4]:
#UNet ckpt path 
ckpt_unet= "Training_LDM.models.components.unet.DownscalingUnetLightning_checkpoint.ckpt"
ckpt_vae= "Training_LDM.models.ae_module.AutoencoderKL_checkpoint.ckpt"
ckpt_ldm= "LDM_checkpoint.ckpt"

Instantiating the UNet

In [5]:
model_UNet= DownscalingUnetLightning.load_from_checkpoint(ckpt_unet,
                                                          in_ch=5,
    out_ch=4,
    features=[64,128,256,512],
    channel_names=["precip", "temp", "temp_min", "temp_max"]
)

Instantiating VAE

In [ ]:
encoder = SimpleConvEncoder(in_dim=4, levels=2, min_ch=16, ch_mult=4)
decoder = SimpleConvDecoder(in_dim=64, levels=2, min_ch=16)

TypeError: __init__() got an unexpected keyword argument 'out_ch'

VAE decoder needed to be softcoded to accept 4 output ch!!

In [10]:
import inspect
print(inspect.signature(SimpleConvDecoder.__init__))

(self, in_dim=1, levels=2, min_ch=16, ch_mult: int = 4)


In [8]:
unet_ckpt = torch.load(ckpt_unet, map_location="cpu")

In [7]:
model_VAE = AutoencoderKL.load_from_checkpoint(
    ckpt_vae,
    encoder=encoder,
    decoder=decoder,
    kl_weight=0.01,
    ae_flag="residual",
    unet_regr=model_UNet,
    strict=False 
)



/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/.micromamba/envs/diffscaler/lib/python3.9/site-packages/lightning/pytorch/core/saving.py:191: Found keys that are in the model state dict but not in the checkpoint: ['unet_regr.unet.e1.conv.conv.0.weight', 'unet_regr.unet.e1.conv.conv.1.weight', 'unet_regr.unet.e1.conv.conv.1.bias', 'unet_regr.unet.e1.conv.conv.1.running_mean', 'unet_regr.unet.e1.conv.conv.1.running_var', 'unet_regr.unet.e1.conv.conv.3.weight', 'unet_regr.unet.e1.conv.conv.4.weight', 'unet_regr.unet.e1.conv.conv.4.bias', 'unet_regr.unet.e1.conv.conv.4.running_mean', 'unet_regr.unet.e1.conv.conv.4.running_var', 'unet_regr.unet.e2.conv.conv.0.weight', 'unet_regr.unet.e2.conv.conv.1.weight', 'unet_regr.unet.e2.conv.conv.1.bias', 'unet_regr.unet.e2.conv.conv.1.running_mean', 'unet_regr.unet.e2.conv.conv.1.running_var', 'unet_regr.unet.e2.conv.conv.3.weight', 'unet_regr.unet.e2.conv.conv.4.weight', 'unet_regr.unet.e2.conv.conv.4.bias', 'unet_r

In [8]:
print(unet_ckpt["state_dict"].keys())
print(model_VAE.unet_regr.state_dict().keys())

odict_keys(['unet.e1.conv.conv.0.weight', 'unet.e1.conv.conv.1.weight', 'unet.e1.conv.conv.1.bias', 'unet.e1.conv.conv.1.running_mean', 'unet.e1.conv.conv.1.running_var', 'unet.e1.conv.conv.1.num_batches_tracked', 'unet.e1.conv.conv.3.weight', 'unet.e1.conv.conv.4.weight', 'unet.e1.conv.conv.4.bias', 'unet.e1.conv.conv.4.running_mean', 'unet.e1.conv.conv.4.running_var', 'unet.e1.conv.conv.4.num_batches_tracked', 'unet.e2.conv.conv.0.weight', 'unet.e2.conv.conv.1.weight', 'unet.e2.conv.conv.1.bias', 'unet.e2.conv.conv.1.running_mean', 'unet.e2.conv.conv.1.running_var', 'unet.e2.conv.conv.1.num_batches_tracked', 'unet.e2.conv.conv.3.weight', 'unet.e2.conv.conv.4.weight', 'unet.e2.conv.conv.4.bias', 'unet.e2.conv.conv.4.running_mean', 'unet.e2.conv.conv.4.running_var', 'unet.e2.conv.conv.4.num_batches_tracked', 'unet.e3.conv.conv.0.weight', 'unet.e3.conv.conv.1.weight', 'unet.e3.conv.conv.1.bias', 'unet.e3.conv.conv.1.running_mean', 'unet.e3.conv.conv.1.running_var', 'unet.e3.conv.conv.1.

In [9]:
#Key mismatch, striiping the prefix unet_regr from the keys of the Unet ckpt for compatibility

Crucial else the following ckpts wont work!!!!!!

In [10]:
remapped_state_dict={}
for k,v in unet_ckpt["state_dict"].items():
    if k.startswith("unet_regr."):
        new_key=k.replace("unet_regr.","unet.")
    else:
        new_key=k
    remapped_state_dict[new_key]=v

Following code will not give a key mismatch error 

In [11]:
model_VAE = AutoencoderKL.load_from_checkpoint(
    ckpt_vae,
    encoder=encoder,
    decoder=decoder,
    kl_weight=0.01,
    ae_flag="residual",
    unet_regr=model_UNet,
    strict=False
)

unet_ckpt = torch.load(ckpt_unet, map_location="cpu")
model_VAE.unet_regr.load_state_dict(remapped_state_dict, strict=False)

<All keys matched successfully>

LDM ckpt loading 

In [12]:

#instantiating denoiser within the LDM
denoiser = UNetModel(
    in_channels=32,  # adjust to your latent dim
    out_channels=32,
    model_channels=64,
    num_res_blocks=2,
    attention_resolutions=[1, 2, 4],
    context_ch=None,
    channel_mult=[1, 2, 4, 4],
    conv_resample=True,
    dims=2,
    use_fp16=False,
    num_heads=4
)



In [13]:
dm_ckpt = torch.load(ckpt_ldm, map_location="cpu")


In [18]:
#instantiating LDM
model_LDM = LatentDiffusion(
    denoiser=denoiser,
    autoencoder=model_VAE,
    timesteps=1000,
    beta_schedule="linear",
    loss_type="l2",
    use_ema=True,
    lr=1e-4,
    lr_warmup=0,
    linear_start=1e-4,
    linear_end=2e-2,
    cosine_s=8e-3,
    parameterization="eps"
)


In [19]:
#The keys within LDM ckpt also have to be renamed for compatibiltiy 

In [20]:
remapped_ldm_state_dict = {}
for k, v in dm_ckpt["state_dict"].items():
    # Remove both 'autoencoder.unet_regr.unet.' and 'autoencoder.unet_regr.' prefixes
    if k.startswith("autoencoder.unet_regr.unet."):
        new_key = "autoencoder.unet." + k[len("autoencoder.unet_regr.unet."):]
    elif k.startswith("autoencoder.unet_regr."):
        new_key = "autoencoder.unet." + k[len("autoencoder.unet_regr."):]
    else:
        new_key = k
    remapped_ldm_state_dict[new_key] = v

In [21]:
model_LDM.load_state_dict(remapped_ldm_state_dict, strict=False)

_IncompatibleKeys(missing_keys=['autoencoder.unet_regr.unet.e1.conv.conv.0.weight', 'autoencoder.unet_regr.unet.e1.conv.conv.1.weight', 'autoencoder.unet_regr.unet.e1.conv.conv.1.bias', 'autoencoder.unet_regr.unet.e1.conv.conv.1.running_mean', 'autoencoder.unet_regr.unet.e1.conv.conv.1.running_var', 'autoencoder.unet_regr.unet.e1.conv.conv.3.weight', 'autoencoder.unet_regr.unet.e1.conv.conv.4.weight', 'autoencoder.unet_regr.unet.e1.conv.conv.4.bias', 'autoencoder.unet_regr.unet.e1.conv.conv.4.running_mean', 'autoencoder.unet_regr.unet.e1.conv.conv.4.running_var', 'autoencoder.unet_regr.unet.e2.conv.conv.0.weight', 'autoencoder.unet_regr.unet.e2.conv.conv.1.weight', 'autoencoder.unet_regr.unet.e2.conv.conv.1.bias', 'autoencoder.unet_regr.unet.e2.conv.conv.1.running_mean', 'autoencoder.unet_regr.unet.e2.conv.conv.1.running_var', 'autoencoder.unet_regr.unet.e2.conv.conv.3.weight', 'autoencoder.unet_regr.unet.e2.conv.conv.4.weight', 'autoencoder.unet_regr.unet.e2.conv.conv.4.bias', 'autoen

In [22]:
#Expected vs real key 
print(model_LDM.state_dict().keys())
print(remapped_ldm_state_dict.keys())

odict_keys(['betas', 'alphas_cumprod', 'alphas_cumprod_prev', 'sqrt_alphas_cumprod', 'sqrt_one_minus_alphas_cumprod', 'denoiser.time_embed.0.weight', 'denoiser.time_embed.0.bias', 'denoiser.time_embed.2.weight', 'denoiser.time_embed.2.bias', 'denoiser.input_blocks.0.0.weight', 'denoiser.input_blocks.0.0.bias', 'denoiser.input_blocks.1.0.in_layers.2.weight', 'denoiser.input_blocks.1.0.in_layers.2.bias', 'denoiser.input_blocks.1.0.emb_layers.1.weight', 'denoiser.input_blocks.1.0.emb_layers.1.bias', 'denoiser.input_blocks.1.0.out_layers.3.weight', 'denoiser.input_blocks.1.0.out_layers.3.bias', 'denoiser.input_blocks.2.0.in_layers.2.weight', 'denoiser.input_blocks.2.0.in_layers.2.bias', 'denoiser.input_blocks.2.0.emb_layers.1.weight', 'denoiser.input_blocks.2.0.emb_layers.1.bias', 'denoiser.input_blocks.2.0.out_layers.3.weight', 'denoiser.input_blocks.2.0.out_layers.3.bias', 'denoiser.input_blocks.3.0.op.weight', 'denoiser.input_blocks.3.0.op.bias', 'denoiser.input_blocks.4.0.in_layers.2.w

Extra key above, rest matching, not giving an issue if strict=False

In [31]:
dm = DownscalingDataModule(
    train_input={
        "precip": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/RhiresD_input_train_chronological_scaled.nc",
        "temp": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TabsD_input_train_chronological_scaled.nc",
        "temp_min": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TminD_input_train_chronological_scaled.nc",
        "temp_max": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TmaxD_input_train_chronological_scaled.nc"
    },
    train_target={
        "precip": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/RhiresD_target_train_chronological_scaled.nc",
        "temp": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TabsD_target_train_chronological_scaled.nc",
        "temp_min": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TminD_target_train_chronological_scaled.nc",
        "temp_max": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TmaxD_target_train_chronological_scaled.nc"
    },
    elevation="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/elevation.tif",
    batch_size=32,
    num_workers=4,
    preprocessing={
        "variables": {
            "input": {
                "precip": "RhiresD",
                "temp": "TabsD",
                "temp_min": "TminD",
                "temp_max": "TmaxD"
            },
            "target": {
                "precip": "RhiresD",
                "temp": "TabsD",
                "temp_min": "TminD",
                "temp_max": "TmaxD"
            }
        },
        "preprocessing": {
            "nan_to_num": True,
            "nan_value": 0.0
        }
    }
)

dm.setup()


In [33]:
train_loader = dm.train_dataloader()
train_batch = next(iter(train_loader))
train_inputs, train_targets = train_batch

# random timestep from the batch
idx = random.randint(0, train_inputs.shape[0] - 1)
input_sample = train_inputs[idx].unsqueeze(0)  #(1, channels, H, W)
target_sample = train_targets[idx].unsqueeze(0)

In [34]:
# Pass through UNet
with torch.no_grad():
    unet_out = model_UNet(input_sample)

print("UNet output shape:", unet_out.shape)


UNet output shape: torch.Size([1, 4, 240, 370])


In [35]:
# Pass through VAE (encode and decode)
with torch.no_grad():
    latent_moments = model_VAE.encode(target_sample)
    latent = latent_moments[0]  # usually mean
    vae_recon = model_VAE.decode(latent)
print("VAE reconstruction shape:", vae_recon.shape)

VAE reconstruction shape: torch.Size([1, 16, 240, 368])


In [ ]:
# through LDM denoiser 
timesteps = torch.tensor([2], dtype=torch.long)
with torch.no_grad():
    denoised = model_LDM.denoiser(latent, timesteps)

print("Denoised latent shape:", denoised.shape)

Denoised latent shape: torch.Size([1, 32, 60, 92])


In [ ]:
#Decodign after denoising 
with torch.no_grad():
    reconstructed = model_VAE.decode(denoised)
print("Reconstructed output shape:", reconstructed.shape) #Shoukld be 4 channels, got 16

Reconstructed output shape: torch.Size([1, 16, 240, 368])
